In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
spark.conf.set("sprk.sql.parquet.vorder.enabled","true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, 6ff04c73-4cf6-424c-b184-5d4de5a36cc7, 4, Finished, Available)

Lees de data uit de RAW folder en toon de eerste 10 records

In [3]:
df = spark.read.format("parquet").load('Files/TCPH-RAW/Customers/*')
display(df.limit(10))

StatementMeta(, 6ff04c73-4cf6-424c-b184-5d4de5a36cc7, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, a0e19bd2-c894-4d6e-b8d0-068e4112655d)

Lees data uit de RAW folder, een specifieke file en toon de eerste 10 records.

In [4]:
df = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/orders.tbl_00000.1.parquet')
display(df.limit(100))

StatementMeta(, 6ff04c73-4cf6-424c-b184-5d4de5a36cc7, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 11fa88c7-4569-43fc-a585-19371a4627f6)

Bouw een Feit tabel gebaseerd op de orders tabel, voeg drie kolommen toe met Jaar, Kwartaal en Maand en schrijf het weg als tabel. File partitionering op basis van Jaar en kwartaal

In [5]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'fact_order'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/*')
df = df.withColumn('Year', year(col("o_orderdate")))
df = df.withColumn('Quarter', quarter(col("o_orderdate")))
df = df.withColumn('Month', month(col("o_orderdate")))

df.write.mode("overwrite").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)


StatementMeta(, 6ff04c73-4cf6-424c-b184-5d4de5a36cc7, 7, Finished, Available)

In [6]:
df = spark.sql("SELECT Year, Quarter, Month, COUNT(o_orderkey) FROM FabricLakehouse.fact_order group by Year, Quarter, Month order by Year, Quarter, Month")
display(df)

StatementMeta(, 6ff04c73-4cf6-424c-b184-5d4de5a36cc7, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2814dba2-fd30-4c93-94d9-3e817221a679)

In [7]:
table_name = 'dim_customer'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Customers/*')

df.write.mode("overwrite").format("delta").save("Tables/" + table_name)

StatementMeta(, 6ff04c73-4cf6-424c-b184-5d4de5a36cc7, 9, Finished, Available)

In [8]:
table_name = 'dim_nation'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Nation/*')

df.write.mode("overwrite").format("delta").save("Tables/" + table_name)

StatementMeta(, 6ff04c73-4cf6-424c-b184-5d4de5a36cc7, 10, Finished, Available)

In [ ]:
table_name = 'dim_region'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Region/*')

df.write.mode("overwrite").format("delta").save("Tables/" + table_name)

In [ ]:
from pyspark.sql.functions import col, year, month

df = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/*')
df = df.withColumn('Year', year(col("o_orderdate")))
df = df.withColumn('Month', month(col("o_orderdate")))
df_filtered = df.filter(df.o_orderkey <= 100)
df_subset = df_filtered.select(df.o_custkey, df.Year, df.Month)
df_count = df_subset.groupby([df_subset.o_custkey, df_subset.Year,df_subset.Month]).count()
df_count.sort(col("count").desc()).show(truncate=False)

StatementMeta(, , , Cancelled, )

In [ ]:
from pyspark.sql.functions import col, year, month

df = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/*')
df = df.withColumn('Year', year(col("o_orderdate")))
df = df.withColumn('Month', month(col("o_orderdate")))
df_filtered = df.filter(df.o_orderdate <= '1992-02-01')
df_subset = df_filtered.select(df.o_custkey, df.Year, df.Month)
df_subset.groupby([df_subset.o_custkey, df_subset.Year,df_subset.Month]).count()
df_count.sort(col("count").desc()).show(truncate=False)

StatementMeta(, , , Cancelled, )

In [ ]:
from pyspark.sql.functions import col, year, month, asc, desc

df = spark.read.format("parquet").load('Tables/fact_order/Year*/*')
df = df.withColumn('Year', year(col("o_orderdate")))
df = df.withColumn('Month', month(col("o_orderdate")))
df_filtered = df.filter(df.o_orderdate <= '1992-02-01')
df_subset = df_filtered.select(df.o_custkey, df.Year, df.Month)
df_count = df_subset.groupby([df_subset.o_custkey, df_subset.Year,df_subset.Month]).count()
df_count.sort(col("count").desc()).show()

StatementMeta(, , , Cancelled, )

In [ ]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'dm_orderline'

dfline = spark.read.format("parquet").load('Files/TCPH-RAW/LineItems/*')
dfcust = spark.read.format("parquet").load('Files/TCPH-RAW/Customers/*')
dfnation = spark.read.format("parquet").load('Files/TCPH-RAW/Nation/*')
dfregion = spark.read.format("parquet").load('Files/TCPH-RAW/Region/*')
dforder = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/*')

dforder = dforder.withColumn('Year', year(col("o_orderdate")))
dforder = dforder.withColumn('Quarter', quarter(col("o_orderdate")))
dforder = dforder.withColumn('Month', month(col("o_orderdate")))

dfordercols = dforder.select(dforder.o_orderkey, dforder.o_custkey, dforder.Year, dforder.Quarter, dforder.Month)
dflinecols = dfline.select(dfline.l_orderkey, dfline.l_linenumber, dfline.l_partkey, dfline.l_quantity, dfline.l_extendedprice, dfline.l_tax )
dfcustcols = dfcust.select(dfcust.c_custkey, dfcust.c_nationkey)
dfnationcols = dfnation.select(dfnation.n_nationkey, dfnation.n_regionkey, dfnation.n_name)
dfregioncols = dfregion.select(dfregion.r_regionkey, dfregion.r_name)

dforderline = dfordercols.join(dflinecols,dfordercols.o_orderkey == dflinecols.l_orderkey, how='Inner')
dforderlinecust = dforderline.join(dfcustcols, dforderline.o_custkey == dfcustcols.c_custkey, how='Inner')
dforderlinecustnation = dforderlinecust.join(dfnationcols, dforderlinecust.c_nationkey == dfnationcols.n_nationkey, how='Inner')
dftotal = dforderlinecustnation.join(dfregioncols, dforderlinecustnation.n_regionkey == dfregioncols.r_regionkey, how='Inner')

dftotal.write.mode("overwrite").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)

StatementMeta(, , , Cancelled, )

In [ ]:
df = spark.sql("SELECT COUNT(1) FROM FabricLakehouse.dm_orderline ")
display(df)

StatementMeta(, , , Waiting, )

In [4]:
%%sql
DESCRIBE HISTORY dim_region

StatementMeta(, fc5a372a-93f1-4ffe-bc7c-8bf1f17eabbe, 6, Finished, Available)

<Spark SQL result set with 2 rows and 15 fields>

In [3]:
%%sql
OPTIMIZE dim_region 

StatementMeta(, fc5a372a-93f1-4ffe-bc7c-8bf1f17eabbe, 5, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>

In [5]:
%%sql
OPTIMIZE fact_order

StatementMeta(, fc5a372a-93f1-4ffe-bc7c-8bf1f17eabbe, 7, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>